# Intelligent AI ChatBot in Python (NeuralNine)

In [35]:
# Intelligent AI ChatBot in Python
# this is going to dyamically understand and respond to greetings
# but the responses will be static

# our files will be : intents.json, responses.json

## TRAINING CHATBOT

In [1]:
# imports
import random
import json
import pickle   # for serialization
import numpy as np
import sqlite3

import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [3]:
# getting intents from database
conn = sqlite3.connect("intents.sqlite")
cur = conn.cursor()

intents = {}

In [6]:
sql_statement = '''
SELECT t.id, t.name FROM tags AS t;
'''
intents = list(map(lambda x: {"tag": x[1]}, cur.execute(sql_statement).fetchall()))
print(intents)
# for i in cur.execute(sql_statement).fetchall():
#     print(i)

[{'tag': 'greetings'}, {'tag': 'goodbye'}, {'tag': 'age'}, {'tag': 'name'}, {'tag': 'shop'}, {'tag': 'hours'}, {'tag': 'crop'}, {'tag': 'stocks'}]


In [15]:
for idx, i in enumerate(intents):
    tag = i['tag']

    sql_pat = '''
    SELECT t.id, t.name, p.name FROM tags AS t JOIN patterns AS p ON t.id == p.tag_id WHERE t.name == '{}';
    '''.format(tag)

    sql_res = '''
    SELECT t.id, t.name, r.name FROM tags AS t JOIN responses AS r ON t.id == r.tag_id WHERE t.name == '{}';
    '''.format(tag)

    intents[idx]['pattern'] = list(map(lambda x: x[2], cur.execute(sql_pat).fetchall()))
    intents[idx]['responses'] = list(map(lambda x: x[2], cur.execute(sql_res).fetchall()))

{'tag': 'greetings', 'pattern': ['hello', 'hey', 'hi', 'good day', 'good morning'], 'responses': ['Hello', 'What can I do for you?']}
{'tag': 'goodbye', 'pattern': ['cya', 'see you later', "I'm leaving", 'goodbye', 'have a good day', 'I am leaving', 'bye', 'see ya'], 'responses': ['Goodbye', 'Nice to have met you']}
{'tag': 'age', 'pattern': ['how old', 'how old is florian', 'what is your age', 'how old are you?', 'age?'], 'responses': ['My owner is 21 years old!', '21 years!', '21']}
{'tag': 'name', 'pattern': ['What is your name', 'what should I call you?', 'what is your name', 'who are you', 'can you tell me your name'], 'responses': ['You can call me Neural!', "I'm neural", "I'm Neural, The assistant"]}
{'tag': 'shop', 'pattern': ["I'd like to buy something", 'What are your products?', 'what do you recommend?', 'What are you selling?'], 'responses': ['We sell books! Lots of them.']}
{'tag': 'hours', 'pattern': ['When are you guys open?', 'What are your hours', 'hours of operation',

In [20]:
for i in intents:
    for j in i.keys():
        print(i[j])
    print()

intents = {"intents": intents}
print(intents['intents'][3]['tag'])

greetings
['hello', 'hey', 'hi', 'good day', 'good morning']
['Hello', 'What can I do for you?']

goodbye
['cya', 'see you later', "I'm leaving", 'goodbye', 'have a good day', 'I am leaving', 'bye', 'see ya']
['Goodbye', 'Nice to have met you']

age
['how old', 'how old is florian', 'what is your age', 'how old are you?', 'age?']
['My owner is 21 years old!', '21 years!', '21']

name
['What is your name', 'what should I call you?', 'what is your name', 'who are you', 'can you tell me your name']
['You can call me Neural!', "I'm neural", "I'm Neural, The assistant"]

shop
["I'd like to buy something", 'What are your products?', 'what do you recommend?', 'What are you selling?']
['We sell books! Lots of them.']

hours
['When are you guys open?', 'What are your hours', 'hours of operation', 'price', 'value', 'mrp']
['24/7']

crop
['What is the price of maize', 'How much is Wheat worth', 'selling price of rice']
['[Unavailable]']

stocks
['what stocks do I own?', 'how are my shares?', 'wha

### *loading our intents*

In [37]:
# loading file
from google.colab import files
uploaded = files.upload()

Saving intents_eng.json to intents_eng (3).json


In [38]:
print(uploaded)

{'intents_eng.json': b'{\r\n    "intents": [\r\n        {\r\n            "tag": "greetings",\r\n            "pattern": ["hello", "hey", "hi", "good day", "good morning"],\r\n            "responses": ["Hello", "What can I do for you?"]\r\n        },  \r\n        {\r\n            "tag": "goodbye",\r\n            "pattern": ["cya", "see you later", "I\'m leaving", "goodbye", "have a good day", "I am leaving", "bye", "see ya"],\r\n            "responses": ["Goodbye", "Nice to have met you"]\r\n        },  \r\n        {\r\n            "tag": "age",\r\n            "pattern": ["how old", "how old is florian", "what is your age", "how old are you?", "age?"],\r\n            "responses": ["My owner florian is 21 years old!", "21 years!", "21"]\r\n        },  \r\n        {\r\n            "tag": "name",\r\n            "pattern": ["What is your name", "what should I call you?", "what is your name", "who are you", "can you tell me your name"],\r\n            "responses": ["You can call me Neural!", 

In [40]:
intents =  json.loads(uploaded['intents_eng.json'])
print(intents['intents'][3]['tag'])

name


### *Creating intermediate Data*

In [21]:
# creatig useful lists
words = []
classes = []
docs = []

ignore_letters = ["!", ".", ",", "?"]

# iterating over intents
for intent in intents['intents']:
  for pat in intent['pattern']:
    # tokenizing and adding tokens to the word list
    word_list = nltk.word_tokenize(pat)
    if word_list is not None:
      words.extend(word_list)

      # appending 
      docs.append((word_list, intent['tag']))

      if intent['tag'] not in classes:
        classes.append(intent['tag'])

In [22]:
print(words)

['hello', 'hey', 'hi', 'good', 'day', 'good', 'morning', 'cya', 'see', 'you', 'later', 'I', "'m", 'leaving', 'goodbye', 'have', 'a', 'good', 'day', 'I', 'am', 'leaving', 'bye', 'see', 'ya', 'how', 'old', 'how', 'old', 'is', 'florian', 'what', 'is', 'your', 'age', 'how', 'old', 'are', 'you', '?', 'age', '?', 'What', 'is', 'your', 'name', 'what', 'should', 'I', 'call', 'you', '?', 'what', 'is', 'your', 'name', 'who', 'are', 'you', 'can', 'you', 'tell', 'me', 'your', 'name', 'I', "'d", 'like', 'to', 'buy', 'something', 'What', 'are', 'your', 'products', '?', 'what', 'do', 'you', 'recommend', '?', 'What', 'are', 'you', 'selling', '?', 'When', 'are', 'you', 'guys', 'open', '?', 'What', 'are', 'your', 'hours', 'hours', 'of', 'operation', 'price', 'value', 'mrp', 'What', 'is', 'the', 'price', 'of', 'maize', 'How', 'much', 'is', 'Wheat', 'worth', 'selling', 'price', 'of', 'rice', 'what', 'stocks', 'do', 'I', 'own', '?', 'how', 'are', 'my', 'shares', '?', 'what', 'companies', 'am', 'I', 'invest

### *Lemmatizing*

In [23]:
# lemmatizing our words
lemmatizer = WordNetLemmatizer()
  
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]

# removing duplicates
words = sorted(set(words))
classes = sorted(set(classes))

LookupError: 
**********************************************************************
  Resource [93momw-1.4[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('omw-1.4')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/omw-1.4[0m

  Searched in:
    - 'C:\\Users\\Angad Sandhu/nltk_data'
    - 'c:\\Users\\Angad Sandhu\\AppData\\Local\\Programs\\Python\\Python39\\nltk_data'
    - 'c:\\Users\\Angad Sandhu\\AppData\\Local\\Programs\\Python\\Python39\\share\\nltk_data'
    - 'c:\\Users\\Angad Sandhu\\AppData\\Local\\Programs\\Python\\Python39\\lib\\nltk_data'
    - 'C:\\Users\\Angad Sandhu\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [44]:
print(words)

["'d", "'m", 'How', 'I', 'What', 'Wheat', 'When', 'a', 'age', 'am', 'are', 'buy', 'bye', 'call', 'can', 'company', 'cya', 'day', 'do', 'doing', 'florian', 'good', 'goodbye', 'guy', 'have', 'hello', 'hey', 'hi', 'hour', 'how', 'in', 'invested', 'is', 'later', 'leaving', 'like', 'maize', 'me', 'morning', 'mrp', 'much', 'my', 'name', 'of', 'old', 'open', 'operation', 'own', 'price', 'product', 'recommend', 'rice', 'see', 'selling', 'share', 'should', 'something', 'stock', 'tell', 'the', 'to', 'value', 'what', 'who', 'worth', 'ya', 'you', 'your']


### *Saving Data*

In [45]:
# adding all info to file
pickle.dump(words, open("words.pkl", 'wb'))
pickle.dump(classes, open("classes.pkl", 'wb'))
pickle.dump(docs, open("docs.pkl", 'wb'))

### *Creattig Training Data*

In [46]:
# representing words as numeric data using bag of words
# so that we could use this in a neural network
training = []
output_empty = [0]*len(classes)

for doc in docs:
  bag = []
  word_patterns = doc[0]
  word_patterns = [lemmatizer.lemmatize(word) for word in word_patterns]

  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)

  # print(output_row)
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1
  training.append([bag, output_row])

### *Pre-Processing*

In [47]:
# shufling our data
random.shuffle(training)
training = np.array(training)

print(training.shape)


# spliting to X and y values
train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

# print(np.asarray(train_x))
print(train_y.shape)

(40, 2)
(40, 8)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


### *Building Neural Net*

In [48]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]), ), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))

In [49]:
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.09, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               8832      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 8)                 520       
                                                                 
Total params: 17,608
Trainable params: 17,608
Non-trainable params: 0
_________________________________________________________________


In [50]:
history = model.fit(train_x, train_y, epochs=200, batch_size=4, verbose=1)

Epoch 1/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1196 - accuracy: 0.1500
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0960 - accuracy: 0.1000
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1099 - accuracy: 0.1250
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0401 - accuracy: 0.0750
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0867 - accuracy: 0.1750
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1345 - accuracy: 0.1750
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0757 - accuracy: 0.1500
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 2.0303 - accuracy: 0.1750
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0009 - accuracy: 0.1250
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0028 - accuracy: 0.1500
Epoch 11/

In [51]:
model.save("ai_chatbot.h5", history)

## EXECUTING CHATBOT

In [52]:
# imports
import random
import json
import pickle   # for serialization
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model

# lemmatizing our words
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### *Loading Our Intents*

In [53]:
# loading file
from google.colab import files
uploaded = files.upload()

Saving intents_eng.json to intents_eng (4).json


In [54]:
print(uploaded)

{'intents_eng.json': b'{\r\n    "intents": [\r\n        {\r\n            "tag": "greetings",\r\n            "pattern": ["hello", "hey", "hi", "good day", "good morning"],\r\n            "responses": ["Hello", "What can I do for you?"]\r\n        },  \r\n        {\r\n            "tag": "goodbye",\r\n            "pattern": ["cya", "see you later", "I\'m leaving", "goodbye", "have a good day", "I am leaving", "bye", "see ya"],\r\n            "responses": ["Goodbye", "Nice to have met you"]\r\n        },  \r\n        {\r\n            "tag": "age",\r\n            "pattern": ["how old", "how old is florian", "what is your age", "how old are you?", "age?"],\r\n            "responses": ["My owner florian is 21 years old!", "21 years!", "21"]\r\n        },  \r\n        {\r\n            "tag": "name",\r\n            "pattern": ["What is your name", "what should I call you?", "what is your name", "who are you", "can you tell me your name"],\r\n            "responses": ["You can call me Neural!", 

In [55]:
intents =  json.loads(uploaded['intents_eng.json'])
print(intents['intents'][2]['tag'])

age


### *Loading up our saved files*

In [56]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
docs = pickle.load(open('docs.pkl', 'rb'))

model = load_model("ai_chatbot.h5")

### *Creating Crop Data Data Structure by Querying dataset*

In [ ]:
# load crop data
uploaded = files.upload()
crop_data =  json.loads(uploaded['crop_data_eng.json'])
print(intents['intents'][3]['tag'])

In [57]:
# Getting Crop Data
crop_data = {
    'rice': 10,
    'wheat': 20,
    'maize': 30
}

crop_names = list(crop_data.keys())

### *Creating Helper Functions*



In [70]:
def clean_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

def bag_of_words(sentence):
  sentence_words = clean_sentence(sentence)
  bag = [0] * len(words)

  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1
  
  return np.array(bag)

def predict_class(sentence):
  ERROR_THRESHOLD = 0.25

  bow = bag_of_words(sentence)
  res = model.predict(np.array([bow]))[0]

  results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

  results.sort(key=lambda x: x[1], reverse=True)

  return_list = []
  for r in results:
    return_list.append({ 'intent': classes[r[0]], 'probability': str(r[1]) })

  return return_list

def get_response(intents_list, intents_json):
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
    if i['tag'] == tag:
      result = random.choice(i['responses'])
      break
  return result

def get_crop_data(msg):
  bow = list(msg.split())
  bow = [string.lower() for string in bow]
  print("message words : ", bow)
  print("crops : ", crop_names)

  common_crops = list(set(bow) & set(crop_names))
  print("common crops : ", common_crops)

  if not common_crops:
    return "No Crop Data Found, Please Try again"
  else:
    crop_res = "The price of {} is {}".format(common_crops[0], crop_data[common_crops[0]])
    return crop_res


## Running BOT

In [73]:
while True:
  message = input('Input Message : ')
  ints = predict_class(message)
  print(ints)

  if ints[0]['intent'] == 'crop':
    res = get_crop_data(message)
  else:
    res = get_response(ints, intents)

  print(res)
  print("Output Message : ", res, "\n")